In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


In [2]:
# from fieldnn.utils.layerfn import traverse
# from fieldnn.utils.simulate import get_next_info, get_simulated_tensor_from_fldname

# Prepare Batch

In [3]:
import os
import torch
import pandas as pd
from recfldgrn.datapoint import load_df_data_from_folder
from recfldgrn.utils import convert_relational_list_to_numpy

###################### take this as given
batch_PID_order = ['P1', 'P4', 'P5', 'P6']
######################

TaskTensor_folder = 'data/ProcData/TensorFolder/Task2YearXXX'
print(TaskTensor_folder)

l = sorted([i for i in os.listdir(TaskTensor_folder) if 'Grn' in i])
l

data/ProcData/TensorFolder/Task2YearXXX


['A1C@DT-DTDftGrn',
 'A1C@V-A1CNumeDftGrn',
 'Diag@DT-DTDftGrn',
 'Diag@Value-DiagDftGrn',
 'EC@BasicInfo-BasicDftGrn',
 'EC@DT_min-DTDftGrn',
 'P@age-AgeNumeDftGrn',
 'P@basicInfo-basicInfoDftGrn',
 'PNSectSent@Sentence-Tk@TknzLLMGrn']

In [4]:
def parse_full_recfldgrn(full_recfldgrn):
    # suffix = full_recfldgrn.split('_')[-1]
    recfld = [i for i in full_recfldgrn.split('-') if '@' in i][0]
    rec, fld = recfld.split('@')
    grn_suffix = [i for i in full_recfldgrn.split('-') if 'Grn' in i][0]
    grn, suffix = grn_suffix.split('_')
    prefix_ids = [i for i in full_recfldgrn.split('-') if 'Grn' not in i and '@' not in i]
    return prefix_ids, rec, fld, grn, suffix

In [5]:
# full_recfldgrn = 'P@age-AgeNumeDftGrn'
# parse_full_recfldgrn(full_recfldgrn)

In [6]:
recfldgrn_list = [
                  'P@age-AgeNumeDftGrn',
                  'A1C@V-A1CNumeDftGrn',
                  'Diag@DT-DTDftGrn',
                  'Diag@Value-DiagDftGrn',
                  'PNSectSent@Sentence-Tk@TknzLLMGrn'
                ]

In [7]:
batch_rfg = {}

for recfldgrn in recfldgrn_list:
    
    # (1) get tensor_folder
    tensor_folder = os.path.join(TaskTensor_folder, recfldgrn)

    # (2) get df_Pat and full_recfldgrn
    df_Pat = load_df_data_from_folder(tensor_folder).set_index('PID')
    full_recfldgrn = df_Pat.columns[0]
    suffix = full_recfldgrn.split('_')[-1]
    assert recfldgrn in full_recfldgrn

    # (3) load batch: TODO: convert this to DataSet and DataLoader
    df_batch = df_Pat.loc[batch_PID_order]

    # (4) tensor batch as tensor_idx
    new_full_recfldgrn = 'B-' + full_recfldgrn
    values_list = df_batch[full_recfldgrn].to_list()
    suffix = full_recfldgrn.split('_')[-1]
    # print(suffix)
    # print(new_full_recfldgrn)
    D = convert_relational_list_to_numpy(values_list, new_full_recfldgrn, suffix)
    tensor_idx = D[new_full_recfldgrn]
    
    batch_rfg[new_full_recfldgrn] = tensor_idx
    
for k, v in batch_rfg.items(): print(k, v.shape)

B-P@age-AgeNumeDftGrn_wgt (4, 19)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt (4, 25, 1, 37)
B-P-EC-Diag@DT-DTDftGrn_idx (4, 25, 22, 7)
B-P-EC-Diag@Value-DiagDftGrn_idx (4, 25, 22, 3)
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx (4, 25, 1, 14, 121, 11)


# EmbedTensor

In [10]:
from fieldnn.sublayer.expander import Expander_Layer
from fieldnn.configfn.expanderfn import get_expander_para

full_recfldgrn_list = [
                  i for i in batch_rfg
                ]
full_recfldgrn_list

['B-P@age-AgeNumeDftGrn_wgt',
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt',
 'B-P-EC-Diag@DT-DTDftGrn_idx',
 'B-P-EC-Diag@Value-DiagDftGrn_idx',
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx']

In [11]:
full_rfg_to_basicInfo = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) full recfldgrn information
    prefix_ids, rec, fld, grn, suffix = parse_full_recfldgrn(full_recfldgrn)
    recfldgrn = rec + '@' + fld + '-' + grn

    # (2) get vocab information
    fldgrn_folder = 'data/ProcData/FldGrnInfo'
    fullfldgrn_file = os.path.join(fldgrn_folder, rec + '.p')
    df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)

    if 'LLM' in full_recfldgrn:
        tokenizer = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        vocab_tokenizer = tokenizer
        init = tokenizer.name_or_path
    else:
        v2idx = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfldgrn].iloc[0]['Vocab']['v2idx']
        # TODO: also adding padding to v2idx
        vocab_size = len(v2idx)
        # print(vocab_size)
        vocab_tokenizer = vocab_size
        init = 'random'
    
    d = {'vocabsize_tokenizer': vocab_tokenizer, 'init': init}
    full_rfg_to_basicInfo[full_recfldgrn] = d
    
full_rfg_to_basicInfo

{'B-P@age-AgeNumeDftGrn_wgt': {'vocabsize_tokenizer': 19, 'init': 'random'},
 'B-P-EC-A1C@V-A1CNumeDftGrn_wgt': {'vocabsize_tokenizer': 37,
  'init': 'random'},
 'B-P-EC-Diag@DT-DTDftGrn_idx': {'vocabsize_tokenizer': 162, 'init': 'random'},
 'B-P-EC-Diag@Value-DiagDftGrn_idx': {'vocabsize_tokenizer': 801,
  'init': 'random'},
 'B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx': {'vocabsize_tokenizer': BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}),
  'init': 'bert-base-uncased'}}

In [12]:
############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

full_rfg_to_ExpanderConfig = {}
for full_recfldgrn in full_recfldgrn_list:
    
    # determine the nn_name
    if '_idx' in full_recfldgrn and 'LLM' not in full_recfldgrn:
        nn_name = 'CatEmbeddingLayer'
    elif '_idx' in full_recfldgrn and 'LLM' in full_recfldgrn:
        nn_name = 'LLMEmbeddingLayer'
    elif '_wgt' in full_recfldgrn:
        nn_name = 'NumEmbeddingLayer'
    else:
        raise ValueError(f'full_recfldgrn is not correct "{full_recfldgrn}"')
    
    d = full_rfg_to_basicInfo[full_recfldgrn]
    expander_para = get_expander_para(full_recfldgrn, 
                                       nn_name,
                                       embed_size, d['vocabsize_tokenizer'], d['init'], 
                                       process)
    # print(expander_para)
    output_recfld = full_recfldgrn.split('Grn')[0]
    
    full_rfg_to_ExpanderConfig[full_recfldgrn] = {
        'full_recfldgrn': full_recfldgrn,
        'output_recfld': output_recfld,
        'expander_para': expander_para,
    }
    
# full_rfg_to_ExpanderConfig

## init model

In [13]:
FullRFG_2_Embed = {}

for full_recfldgrn, ExpanderConfig in full_rfg_to_ExpanderConfig.items():
    
    FullRFG_2_Embed[full_recfldgrn] = Expander_Layer(**ExpanderConfig)
    print(full_recfldgrn)
    print(FullRFG_2_Embed[full_recfldgrn])
    

B-P@age-AgeNumeDftGrn_wgt
Expander_Layer(
  (Embed): NumEmbeddingLayer(
    (embedding): Embedding(19, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P-EC-A1C@V-A1CNumeDftGrn_wgt
Expander_Layer(
  (Embed): NumEmbeddingLayer(
    (embedding): Embedding(37, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P-EC-Diag@DT-DTDftGrn_idx
Expander_Layer(
  (Embed): CatEmbeddingLayer(
    (embedding): Embedding(162, 128, padding_idx=0)
  )
  (postprocess): ModuleDict(
    (activator): GELU(approximate='none')
    (dropout): Dropout(p=0.5, inplace=False)
    (layernorm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  )
)
B-P-EC-Diag@Value-DiagDftGrn_idx

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
Expander_Layer(
  (Embed): LLMEmbeddingLayer(
    (LLM): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense):

## prepare input

In [14]:
for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print(full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str: 
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')

    print('\n')

B-P@age-AgeNumeDftGrn_wgt
(4, 19)
torch.Size([4, 19]) <------ holder shape
torch.Size([4, 19]) <------ holder wgt information


B-P-EC-A1C@V-A1CNumeDftGrn_wgt
(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information


B-P-EC-Diag@DT-DTDftGrn_idx
(4, 25, 22, 7)
torch.Size([4, 25, 22, 7]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-Diag@Value-DiagDftGrn_idx
(4, 25, 22, 3)
torch.Size([4, 25, 22, 3]) <------ holder shape
Empty <------ holder wgt information


B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLMGrn_idx
(4, 25, 1, 14, 121, 11)
torch.Size([4, 25, 1, 14, 121, 11]) <------ holder shape
Empty <------ holder wgt information




## I-NN-O

In [15]:
full_recfldgrn_EmbedTensor = {}

for full_recfldgrn in full_recfldgrn_list:
    # (1) get the info_raw from batch_rfg
    info_raw = batch_rfg[full_recfldgrn]
    print('Input:',full_recfldgrn)
    print(info_raw.shape)

    # (2) get the holder (input_idx) and holder_wgt (for nume embedding only)
    if '_idx' in full_recfldgrn:
        holder_wgt = 'Empty'
        holder = torch.LongTensor(info_raw)
    elif '_wgt' in full_recfldgrn:
        holder_wgt = torch.FloatTensor(info_raw)
        # ATTENTION: here holder_wgt could contain zeros in some valid positions.
        holder = torch.ones_like(holder_wgt).cumsum(-1).masked_fill(holder_wgt == 0, 0).long()
    else:
        raise ValueError(f'Invalid suffix "{suffix}"')

    print(holder.shape, '<------ holder shape')
    if type(holder_wgt) == str:  
        print(holder_wgt, '<------ holder wgt information')
    else:
        print(holder_wgt.shape, '<------ holder wgt information')
    
    NN = FullRFG_2_Embed[full_recfldgrn] 
    print(holder.shape, '<----- holder shape')
    output_recfld, holder, info = NN(full_recfldgrn, holder, holder_wgt)
    print('Output:', output_recfld)
    print(info.shape, '<----- info shape')
    full_recfldgrn_EmbedTensor[output_recfld] = info
    print('\n')

Input: B-P@age-AgeNumeDftGrn_wgt
(4, 19)
torch.Size([4, 19]) <------ holder shape
torch.Size([4, 19]) <------ holder wgt information
torch.Size([4, 19]) <----- holder shape
Output: B-P@age-AgeNumeDft
torch.Size([4, 19, 128]) <----- info shape


Input: B-P-EC-A1C@V-A1CNumeDftGrn_wgt
(4, 25, 1, 37)
torch.Size([4, 25, 1, 37]) <------ holder shape
torch.Size([4, 25, 1, 37]) <------ holder wgt information
torch.Size([4, 25, 1, 37]) <----- holder shape
Output: B-P-EC-A1C@V-A1CNumeDft
torch.Size([4, 25, 1, 37, 128]) <----- info shape


Input: B-P-EC-Diag@DT-DTDftGrn_idx
(4, 25, 22, 7)
torch.Size([4, 25, 22, 7]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 22, 7]) <----- holder shape
Output: B-P-EC-Diag@DT-DTDft
torch.Size([4, 25, 22, 7, 128]) <----- info shape


Input: B-P-EC-Diag@Value-DiagDftGrn_idx
(4, 25, 22, 3)
torch.Size([4, 25, 22, 3]) <------ holder shape
Empty <------ holder wgt information
torch.Size([4, 25, 22, 3]) <----- holder shape
Output: B-P-EC-

# Learner Helper Fn

In [16]:
# from fieldnn.utils.layerfn import traverse
# from fieldnn.utils.simulate import get_next_info, get_simulated_tensor_from_fldname

# # leng_mask = info_idx == 0

# import torch

# def get_Layer2Holder(fullname, holder, Ignore_PSN_Layers = ['B', 'P']):
#     # holder = holder
#     d = {}
#     for layername in list(reversed(fullname.split('-'))):
#         if layername in Ignore_PSN_Layers: continue
#         leng_mask = holder == 0
#         leng = (leng_mask == 0).sum(-1)
#         psn_idx = (leng_mask == False).cumsum(-1).masked_fill(leng_mask, 0)
#         d[layername] = {'holder': holder, 
#                         'leng_mask': leng_mask, 
#                         'leng': leng, 
#                         'psn_idx': psn_idx}
#         # d[layername] = holder, psn_idx
#         holder = leng
#     Layer2Hoder = d
#     return Layer2Hoder

In [17]:

# def align_psn_idx(source_layer, current_layer, Layer2Idx, Layer2Holder):
#     if source_layer == current_layer:
#         psn_idx = Layer2Holder[current_layer]['psn_idx']
#         return psn_idx
#     else:
#         source_psn_idx = Layer2Holder[source_layer]['psn_idx']
#         current_leng_mask = Layer2Holder[current_layer]['leng_mask']
#         gaps = Layer2Idx[current_layer] - Layer2Idx[source_layer]
#         # print(gaps)
#         # print(layername)
#         # print(prev_info.shape)
#         # print(leng_mask.shape)
#         # print(leng.shape)
#         # print(psn_idx.shape)
#         shape0 = list(source_psn_idx.shape) + [1] * gaps
#         shape1 = current_leng_mask.shape
#         psn_idx = source_psn_idx.view(*shape0).expand(shape1).masked_fill(current_leng_mask, 0)
#         # print(cpsn_idx.shape)
#         return psn_idx

# Learner Core

## TFMLayer

### Module

In [18]:
import torch
import numpy as np
import torch.nn.functional as F

def _addindent(s_, numSpaces):
    s = s_.split('\n')
    # don't do anything for single-line stuff
    if len(s) == 1:
        return s_
    first = s.pop(0)
    s = [(numSpaces * ' ') + line for line in s]
    s = '\n'.join(s)
    s = first + '\n' + s
    return s

class TFMLayer(torch.nn.Module):
    def __init__(self, 
                 input_size = 200, 
                 output_size = 200, # d_model
                 nhead = 8,
                 num_encoder_layers = 6, # only have encoder part
                 num_decoder_layers = 0, # in default, we don't need decoder part. 
                 dim_feedforward = 2048, 
                 tfm_dropout = 0.1,
                 tfm_activation = 'relu'):
        
        '''https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/transformer.py'''

        super(TFMLayer,self).__init__()
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.input_size = input_size
        self.tfm_input_size = input_size
        self.n_directions = 1
        self.output_size = output_size
        assert output_size % self.n_directions == 0 
        self.hidden_size = int(output_size / self.n_directions)
        assert self.hidden_size == self.tfm_input_size
        # self.psn_size = psn_size 
        
        self.dim_feedforward = dim_feedforward
        
        self.transformer  = torch.nn.Transformer(d_model = self.hidden_size, 
                                                 nhead = nhead,
                                                 num_encoder_layers = self.num_encoder_layers,
                                                 num_decoder_layers = self.num_decoder_layers,
                                                 dim_feedforward = dim_feedforward, 
                                                 dropout = tfm_dropout,
                                                 activation = tfm_activation,
                                                 batch_first = True,
                                                 # src_mask_flag = False, # see all tokens in a sentence 
                                                 # # This IS THE NEW PART. NOT PyTorch.nn.
                                                 ) 


    def forward(self, info, leng_mask):
        info = self.transformer(info, info, 
                                src_key_padding_mask = leng_mask,  
                                tgt_key_padding_mask = leng_mask)
        # for layer in self.postprocess:
        #     info = layer(info)
        return info
    
    
    def __repr__(self):
        # We treat the extra repr like the sub-module, one item per line
        extra_lines = []
        extra_repr = self.extra_repr()
        # empty string will be split into list ['']
        if extra_repr:
            extra_lines = extra_repr.split('\n')
        child_lines = []
        for key, module in self._modules.items():
            mod_str = repr(module)
            mod_str = _addindent(mod_str, 2)
            child_lines.append('(' + key + '): ' + mod_str)
        lines = extra_lines + child_lines

        main_str = self._get_name() + f'LEARNER(TFM): input({self.input_size}), output({self.output_size})'
        lines = [f'(Encoder): EncoderLayer(layers_num={self.num_encoder_layers}, dim_feedforward={self.dim_feedforward})', 
                 f'(Decoder): DecoderLayer(layers_num={self.num_decoder_layers}, dim_feedforward={self.dim_feedforward})']
        main_str += '\n  ' + '\n  '.join(lines) + '\n'
        
        # if lines:
        #     # simple one-liner info, which most builtin Modules will use
        #     if len(extra_lines) == 1 and not child_lines:
        #         main_str += extra_lines[0]
        #     else:
        #         main_str += '\n  ' + '\n  '.join(lines) + '\n'
        # main_str += ')'
        return main_str
    

### Config

In [16]:
tfm_para =  {'input_size': 512,
             'output_size': 512,
             'nhead': 8,
             'num_encoder_layers': 6,
             'num_decoder_layers': 0,
             'dim_feedforward': 2048,
             'tfm_dropout': 0.1,
             'tfm_activation': 'relu'}
tfm_para

{'input_size': 512,
 'output_size': 512,
 'nhead': 8,
 'num_encoder_layers': 6,
 'num_decoder_layers': 0,
 'dim_feedforward': 2048,
 'tfm_dropout': 0.1,
 'tfm_activation': 'relu'}

### Usage

#### recfld information

In [17]:
for recfld, EmbedTensor in full_recfldgrn_EmbedTensor.items():
    print(recfld, EmbedTensor.shape)

B-P-EC-A1C@V-A1CNumeDft torch.Size([4, 25, 1, 37, 128])
B-P-EC-Diag@Value-DiagDft torch.Size([4, 25, 22, 3, 128])
B-P-EC-PN-PNSect-PNSectSent@Sentence-Tk@TknzLLM torch.Size([4, 25, 1, 14, 121, 11, 128])


#### get configuration

#### init model

#### prepare input

In [59]:
tfm_layer = TFMLayer(**tfm_para)
# tfm_layer

In [60]:
tfm_layer

TFMLayerLEARNER(TFM): input(512), output(512)
  (Encoder): EncoderLayer(layers_num=6, dim_feedforward=2048)
  (Decoder): DecoderLayer(layers_num=0, dim_feedforward=2048)

#### I-NN-O

## LinearLayer

### Module

In [62]:
import torch
import math
import torch.nn.functional as F

class LinearLayer(torch.nn.Module):

    def __init__(self, 
                 input_size  = 200, 
                 output_size = 200.
                 ):

        super(LinearLayer, self).__init__()
    
        self.input_size  = input_size
        self.output_size = output_size
        self.linear  = torch.nn.Linear(self.input_size, self.output_size)
        self.init_weights()
            
    def init_weights(self):
        initrange = 0.1
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, info):
        info = self.linear(info)
        return info

### Config

### Usage

#### recfld information 

#### get configuration

#### init model

#### prepare input

#### I-NN-O

# Learner

## Module

In [3]:
import os
import torch
import numpy as np

from fieldnn.nn.tfm import TFMLayer
from fieldnn.nn.linear import LinearLayer
from fieldnn.utils.layerfn import orderSeq, restoreSeq, align_psn_idx, get_Layer2Holder
from fieldnn.utils.parafn import generate_psn_embed_para


class Learner_Layer(torch.nn.Module):
    def __init__(self, input_fullname, output_fullname, learner_layer_para, PSN_Embed_ModuleDict):
        super(Learner_Layer, self).__init__()
        
        # Part 0: Meta
        self.fullname = input_fullname
        
        self.input_fullname = input_fullname
        self.output_fullname = output_fullname
        assert self.input_fullname == self.output_fullname
        
        self.input_size = learner_layer_para['input_size']
        self.output_size = learner_layer_para['output_size']
        self.embed_size = self.input_size
        self.max_leng = learner_layer_para['max_leng']
        
            
        # Part 1: NN
        nn_name, para = learner_layer_para[self.fullname]
        self.nn_name == nn_name
        
        if nn_name.lower() == 'tfm':
            assert self.input_size == self.output_size
            self.Learner = TFMLayer(**para)
            # Part a: PSN embedding
            self.psn_embedding = torch.nn.Embedding(self.max_leng + 1, self.input_size, padding_idx = 0)
            # Part b: EmbedProcess
            self.embedprocess = torch.nn.ModuleDict()
            for method, config in learner_layer_para['embedprocess'].items():
                if method == 'dropout':
                    self.embedprocess[method] = torch.nn.Dropout(**config)
                elif method == 'layernorm':
                    self.embedprocess[method] = torch.nn.LayerNorm(self.output_size, **config)
                else:
                    raise ValueError(f'no avialable embedprocess method {method}')
                    
            self.forward = self.forward_tfm
                
        elif nn_name.lower() == 'linear':
            self.Learner = LinearLayer(**para)
            self.forward = self.forward_lnr
            
        # elif nn_name.lower() == 'cnn':
        #     self.Learner = CNNLayer(**para)
        
        # elif nn_name.lower() == 'rnn':
        #     self.Learner = RNNLayer(**para)
            
        else:
            raise ValueError(f'NN "{nn_name}" is not available')
        
        # Part 2: PostProcess
        self.postprocess = torch.nn.ModuleDict()
        for method, config in learner_layer_para['postprocess'].items():
            if method == 'dropout':
                self.postprocess[method] = torch.nn.Dropout(**config)
            elif method == 'layernorm':
                self.postprocess[method] = torch.nn.LayerNorm(self.output_size, **config)

        self.Ignore_PSN_Layers = learner_layer_para['Ignore_PSN_Layers']
        
    # def get_psn_embed_tensor(self, fullname, holder):
    #     name = fullname.split('-')[-1]
    #     Layer2Idx = {v:idx for idx, v in enumerate(fullname.split('-'))}
    #     Layer2Holder = get_Layer2Holder(fullname, holder, self.Ignore_PSN_Layers)
    #     psn_embed = 0
    #     for source_layer, Embed in self.PSN_Embed_ModuleDict.items():
    #         cpsn_idx = align_psn_idx(source_layer, name, Layer2Idx, Layer2Holder)
    #         psn_embed = psn_embed + Embed(cpsn_idx)
    #     return psn_embed
    
    def reshape(self, info, leng_mask):
        nbs = np.array(info.shape[:-2]).prod()
        ngrn, dim = info.shape[-2:]
        # print(nbs, ngrn, dim)
        
        tmp_info = info.contiguous().view(nbs, ngrn, dim)
        # print(tmp_info.shape)

        tmp_leng_mask = leng_mask.contiguous().view(nbs, ngrn)
        # print(tmp_leng_mask.shape)

        tmp_leng = (tmp_leng_mask == 0).sum(-1)
        # print(tmp_leng.shape)
        
        ord_info,      ord_leng, r_idx = orderSeq(tmp_info, tmp_leng)
        ord_leng_mask, ord_leng, r_idx = orderSeq(tmp_leng_mask, tmp_leng)
        return ord_info, ord_leng_mask, r_idx
    
    def restore(self, ord_info_output, leng_mask, r_idx):
        info_new = restoreSeq(ord_info_output, r_idx)
        output_size = info_new.shape[-1]
        info_output = info_new.view(*list(leng_mask.shape) + [output_size])
        return info_output
        
    def generate_locidx(self, holder):
        leng_mask = holder == 0
        # leng = (leng_mask == 0).sum(-1)
        psn_idx = (leng_mask == False).cumsum(-1).masked_fill(leng_mask, 0)
        return psn_idx
    
    def forward_tfm(self, fullname, holder, info):
        assert self.input_fullname == fullname
        
        # (1) adding psn embed
        psn_locid = self.generate_locidx(holder) #### TODO
        psn_embed = self.psn_embedding(psn_locid)
        for nn, layer in self.embedprocess.items(): 
            psn_embed = layer(psn_embed)
        info = info + psn_embed
        # TODO: process psn_embed? Do we need the further embed process? 
        
        # (2) do the tfm calculation
        leng_mask = holder == 0
        ord_info, ord_leng_mask, r_ix = self.reshape(info, leng_mask)
        ord_info_output = self.Learner(ord_info, ord_leng_mask)
        info = self.restore(ord_info_output, leng_mask, r_ix)
        
        # (3) post-process
        for nn_name, layer in self.postprocess.items():
            info = layer(info)
            
        # we do not change the fullname and holder
        return self.output_fullname, holder, info
    
    
    def forward_lnr(self, fullname, holder, info):
        assert self.input_fullname == fullname
        # (1) learn the info
        info = self.Learner(info)
        
        # (2) do the masked_leng because of non-zero bias
        leng_mask = holder == 0
        info = info.masked_fill(leng_mask.unsqueeze(-1), 0)
    
        # (3) post-process
        for nn_name, layer in self.postprocess.items():
            info = layer(info)
            
        # we do not change the fullname and holder
        return self.output_fullname, holder, info

## Config

## Usage

### recfld informaiton

### get configuration

### init model

### prepare input

### I-NN-O